In [1]:
class Symbol:
  def __init__(self, index, val):
    self.index = index
    self.val = val

  def __str__(self):
    return self.val

class StateMachine:
  def __init__(self, pattern):
    self.pattern = pattern
    self.pi = StateMachine.build_pi(pattern)

  def init_state_cache(self, fsst_symbols):
    self.fsst_symbols = fsst_symbols
    self.cache_state = [-1] * (len(self.pattern) * len(fsst_symbols))

  def build_state_cache(self, fsst_symbols):
    assert self.pattern is not None

    # Init the state cache.
    self.init_state_cache(fsst_symbols)

    for i in range(len(self.pattern)):
      for symbol in fsst_symbols:
        # Set the current state to position `i`.
        self.init_state(i)

        # Simulate.
        self.accept_symbol(symbol)

        # Cache the state we arrived at.
        self.cache_state[i * len(fsst_symbols) + symbol.index] = self.curr_state
        
  def init_state(self, pos=0):
    self.curr_state = pos

  def accept_letter(self, letter):
    while (self.curr_state > 0) and (self.pattern[self.curr_state] != letter):
      self.curr_state = self.pi[self.curr_state - 1]
    
    if self.pattern[self.curr_state] == letter:
      self.curr_state += 1

  def accept_symbol(self, symbol):
    for letter in symbol.val:
      self.accept_letter(letter)

      # Already reached the final state?
      if self.curr_state == len(self.pattern):
        return

  def accept_cached_symbol(self, symbol):
    assert self.fsst_symbols is not None
    assert self.cache_state is not None
    
    # Use the cache.
    self.curr_state = self.cache_state[self.curr_state * len(self.fsst_symbols) + symbol.index]

  def accept_symbol_lazily(self, symbol):
    assert self.fsst_symbols is not None
    assert self.cache_state is not None
    
    # The position in the state cache.
    pos = self.curr_state * len(self.fsst_symbols) + symbol.index

    # Fast path.
    if self.cache_state[pos] != -1:
      self.curr_state = self.cache_state[pos]
      return

    # Otherwise, run. Note: the current state is updated.
    self.accept_symbol(symbol)

    # And cache it.
    self.cache_state[pos] = self.curr_state

  @staticmethod
  # Source: https://www.algopedia.ro/wiki/index.php/Clasele_9-10_lecția_11_-_26_nov_2014.
  def build_pi(P):
    # Init.
    pi = [0] * len(P)

    # Nowhere to go.
    pi[0] = 0

    # Init the state.
    k = 0
    for q in range(1, len(P)):
      while (k > 0) and (P[k] != P[q]):
        k = pi[k - 1]
      
      # Match? Then advance.
      if P[k] == P[q]:
        k += 1
      
      # Store the state.
      pi[q] = k

    return pi
  
  def naive_match(self, text):
    # Init.
    self.init_state()

    for i in range(len(text)):
      if isinstance(text[i], Symbol):
        self.accept_symbol(text[i])
      else:
        self.accept_letter(text[i])
      
      if self.curr_state == len(self.pattern):
        return True
      
    return False
  
  def fast_match(self, text):
    assert self.cache_state is not None

    # Init.
    self.init_state()

    for i in range(len(text)):
      # If we have a symbol, use the state cache.
      if isinstance(text[i], Symbol):
        self.accept_cached_symbol(text[i])
      else:
        self.accept_letter(text[i])
      
      if self.curr_state == len(self.pattern):
        return True
      
    return False
  
  def fast_match_lazily(self, text):
    assert self.cache_state is not None

    # Init.
    self.init_state()

    for i in range(len(text)):
      # If we have a symbol, use the state cache.
      if isinstance(text[i], Symbol):
        self.accept_symbol_lazily(text[i])
      else:
        self.accept_letter(text[i])
      
      if self.curr_state == len(self.pattern):
        return True
      
    return False

In [2]:
import random
import string

random.seed(0)

letter_set = ['a', 'b', 'c']

def generate_random_string(fsst_symbols):
  return [random.choice(fsst_symbols if random.random() < 0.2 else letter_set) for _ in range(20)]

def generate_random_symbol_string():
  return ''.join(random.choices(letter_set, k=8))

def python_match(column_data, pattern):
  ret = []

  # NOTE: These are the raw strings.
  for index, text in enumerate(column_data):
    if pattern in text:
      ret.append(index)
  return ret

def eval_naive_match(column_data, pattern):
  state_machine = StateMachine(pattern)

  ret = []
  for index, text in enumerate(column_data):
    if state_machine.naive_match(text):
      ret.append(index)
  return ret

def eval_fast_match(column_data, pattern):
  state_machine = StateMachine(pattern)
  state_machine.build_state_cache(fsst_symbols)

  ret = []
  for index, text in enumerate(column_data):
    if state_machine.fast_match(text):
      ret.append(index)
  return ret

def eval_fast_match_lazily(column_data, pattern):
  state_machine = StateMachine(pattern)
  state_machine.init_state_cache(fsst_symbols)

  ret = []
  for index, text in enumerate(column_data):
    if state_machine.fast_match_lazily(text):
      ret.append(index)
  return ret

FSST_SIZE = 256
fsst_symbols = [Symbol(i, generate_random_symbol_string()) for i in range(FSST_SIZE)]

N = 100_000
strings = [generate_random_string(fsst_symbols) for _ in range(N)]

raw_strings = [''.join(list(map(str, x))) for x in strings]

for k in range(1, 10):
  pattern = ''.join(random.choices(letter_set, k=k))

  import time
  t0 = time.time_ns()
  ret0 = python_match(raw_strings, pattern)
  t1 = time.time_ns()
  ret1 = eval_naive_match(strings, pattern)
  t2 = time.time_ns()
  ret2 = eval_fast_match(strings, pattern)
  t3 = time.time_ns()
  ret3 = eval_fast_match_lazily(strings, pattern)
  t4 = time.time_ns()

  print(f'patttern= {pattern}:')
  assert ret0 == ret1
  assert ret0 == ret2
  assert ret0 == ret3

  print(f'Python: {(t1 - t0) / 1e6} ms')
  print(f'Naive: {(t2 - t1) / 1e6} ms')
  print(f'Fast: {(t3 - t2) / 1e6} ms')
  print(f'Fast w/ lazy: {(t4 - t3) / 1e6} ms')

patttern= c:
Python: 6.691 ms
Naive: 80.373 ms
Fast: 64.18 ms
Fast w/ lazy: 74.627 ms
patttern= ba:
Python: 11.612 ms
Naive: 265.79 ms
Fast: 157.632 ms
Fast w/ lazy: 155.07 ms
patttern= aaa:
Python: 20.707 ms
Naive: 684.835 ms
Fast: 314.219 ms
Fast w/ lazy: 326.855 ms
patttern= cbca:
Python: 24.576 ms
Naive: 873.008 ms
Fast: 400.302 ms
Fast w/ lazy: 414.227 ms
patttern= babba:
Python: 19.902 ms
Naive: 1012.776 ms
Fast: 453.708 ms
Fast w/ lazy: 469.244 ms
patttern= babcab:
Python: 29.614 ms
Naive: 1103.51 ms
Fast: 489.082 ms
Fast w/ lazy: 535.667 ms
patttern= cbbbaab:
Python: 29.848 ms
Naive: 1156.447 ms
Fast: 500.771 ms
Fast w/ lazy: 518.778 ms
patttern= abababaa:
Python: 19.256 ms
Naive: 1114.732 ms
Fast: 500.323 ms
Fast w/ lazy: 518.352 ms
patttern= baacacaab:
Python: 18.719 ms
Naive: 1140.223 ms
Fast: 503.137 ms
Fast w/ lazy: 520.503 ms
